<a href="https://colab.research.google.com/github/PigStep/furniture-store-extraction/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import numpy as np

#For parsing
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import re
import json

In [100]:
urls_df = pd.read_csv("URL_list.csv")
urls_df.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [101]:
url_series = pd.Series(urls_df["max(page)"]) #To make code easier
url_series.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


# Предобработка данных
Сохраним только открывающиеся ссылки

In [92]:
def check_is_url_parsing(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  try:
      response = requests.get(url, headers=headers, timeout=10)
      response.raise_for_status() #check for html error
      soup = BeautifulSoup(response.text, "html.parser")

      return True

  except requests.exceptions.RequestException as e:
    print(f"Attemp to URL falied: {e}")
    return False

In [205]:
def create_good_urls_csv():
  """Создает файл 'ParsingURL_list.csv' с посещаемыми ссылками"""
  good_urls =[]
  for url in url_series:
    if(check_is_url_parsing(url)):
      good_urls.append(url)

  urls = pd.Series(good_urls, name="url")
  urls.to_csv("ParsingURL_list.csv")
  print("Dataset have been created and saved")

In [95]:
# create_good_urls_csv()

In [168]:
# Пересоздадим очищенную серию
url_series = pd.read_csv("ParsingURL_list.csv")
url_series = pd.Series(url_series["url"])

#Parsing data from sites

# Создадим "идеальную выборку"
Некоторые сайты имеют application json теги для системы поисковиков. Их очень удобно парсить и они почти не имеют шума

In [207]:
def get_parsed_data(url):
    """Получает данные о продукте из JSON-LD разметки страницы"""
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        scripts = soup.find_all("script", type="application/ld+json")
        product = None

        for script in scripts:
            try:
                data = json.loads(script.string)

                if isinstance(data, list):
                    for obj in data:
                        if isinstance(obj, dict) and obj.get("@type") == "Product":
                            product = obj
                            break
                elif isinstance(data, dict) and data.get("@type") == "Product":
                    product = data

                if product:
                    print(product)

                    offers = product.get("offers", {})
                    if isinstance(offers, list):
                        offers = offers[0] if offers else {}
                    price = offers.get("price")

                    # Чтобы предотвратить ошибки типов
                    brand = product.get("brand")
                    brand_name = brand if isinstance(brand, str) else (brand.get("name") if isinstance(brand, dict) else None)

                    return {
                        "url": url,
                        "title": product.get("name"),
                        "description": product.get("description"),
                        "price": price,
                        "brand": brand_name
                    }

            except json.JSONDecodeError:
                continue

        print(f'No product data found in JSON-LD: <a href="{url}">{quote(url)}</a>')
        return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

In [209]:
def get_cleaned_data():
  all_data =[]
  for url in url_series:
    product_data = get_parsed_data(url)

    if(product_data):
      product_data["url"] = url
      all_data.append(product_data)

  # Создаем датафрейм разом
  urls_df = pd.DataFrame(all_data)
  urls_df.to_csv("JSON_train_data.csv")
  print("Dataset created and saved succseffully as 'JSON_train_data.csv'")

get_cleaned_data()


{'@context': 'http://schema.org/', '@type': 'Product', 'url': 'https://www.factorybuys.com.au/products/euro-top-mattress-king', 'name': 'Factory Buys 32cm Euro Top Mattress - King', 'image': '//www.factorybuys.com.au/cdn/shop/files/MATTRESS-0907-K-5672-07.jpg?v=1718172132', 'description': 'Pamper yourself with Factory Buys Euro Top Mattress that will give you a soothing sleep night after night.Incorporating the cutting-edge technology, the lavish construction addresses all elements that will offer you a comfortable and luxurious experience - Euro top padding with high-density foam and an independent coil', 'brand': {'@type': 'Brand', 'name': 'Factory Buys'}, 'sku': 'MATTRESS-0907-K', 'mpn': 'MATTRESS-0907-K', 'aggregateRating': {'@type': 'AggregateRating', 'ratingValue': '4.9', 'reviewCount': 22}, 'offers': {'@type': 'Offer', 'url': 'https://www.factorybuys.com.au/products/euro-top-mattress-king?variant=36449444554', 'itemCondition': 'http://schema.org/NewCondition', 'availability': 'h